# Two-body problems

This module covers solution methods for two-body problems in the **perifocal frame**: 
given some information about an orbit, how can we find the new position and velocity
after some change in true anomaly or after some time?

This will cover two-body problem solutions using Lagrange coefficients,
Kepler problems (involving time), the Kepler problem with universal variables,
and solving the orbital equation of motion numerically from initial conditions.

In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# these lines are only for helping improve the display
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 150
plt.rcParams['savefig.dpi'] = 150

## Two-body problems in perifocal frame

**The problem:** given the initial position vector $\vec{r}_0$ and initial velocity vector $\vec{v}_0$ (the state vectors), 
known at some instant in time, find the new state vectors $\vec{r}$ and $\vec{v}$ after the true anomaly changes by $\Delta \theta$.

:::{figure,myclass} fig-twobody
<img src="../images/two-body-problem.png" alt="Two-body problem diagram" class="bg-primary mb-1" width="400px">

Diagram of two-body problem with state vectors.
:::

Since the state vectors $\vec{r}$ and $\vec{v}$ will always remain in the same two-dimensional plane, 
they must be able to be expressed as linear combinations of
$\vec{r}_0$ and $\vec{v}_0$:

$$
\begin{align}
\vec{r} &= f \vec{r}_0 + g \vec{v}_0 \\
\vec{v} &= \dot{f} \vec{r}_0 + \dot{g} \vec{v}_0 \;,
\end{align}
$$

where $f, g, \dot{f}, \dot{g}$ are the **Lagrange coefficients**, which are constant scalars.

Note that by substituting these into the definition of specific angular momentum $\vec{h}$, 
which must remain constant, we can obtain the **Lagrange identity**:

$$
\begin{align}
\vec{h} = \vec{r}_0 \times \vec{v}_0 &= \vec{r} \times \vec{v} \\
&= \left( f \vec{r}_0 + g \vec{v}_0 \right) \times \left( \dot{f} \vec{r}_0 + \dot{g} \vec{v}_0 \right) \\
&= f \dot{f} \vec{r}_0 \times \vec{r}_0 + f \dot{g} \vec{r}_0 \times \vec{v}_0 + g \dot{f} \vec{v}_0 \times \vec{r}_0 + g \dot{g} \vec{v}_0 \times \vec{v}_0 \\
&= f \dot{g} \vec{r}_0 \times \vec{v}_0 + g \dot{f} \vec{v}_0 \times \vec{r}_0 \\
&= f \dot{g} \vec{r}_0 \times \vec{v}_0 - g \dot{f} \vec{r}_0 \times \vec{v}_0 \\
\vec{r}_0 \times \vec{v}_0 &= \left( f \dot{g} - g \dot{f} \right) \vec{r}_0 \times \vec{v}_0 \\
\therefore f \dot{g} - g \dot{f} &= 1 \;,
\end{align}
$$

which provides a potentially helpful relationship between the coefficients—given three of them, you can easily find the fourth.

### Lagrange coefficients

Via a complicated derivation, we can find closed-form expressions for the Lagrange coefficients:

$$
\begin{align}
f &= 1 - \frac{\mu r}{h^2} \left( 1 - \cos \Delta \theta \right) \\
g &= \frac{r r_0}{h} \sin \Delta \theta \\
\dot{f} &= \frac{\mu}{h} \frac{1 - \cos \Delta \theta}{\sin \Delta \theta} \left[ \frac{\mu}{h^2} \left( 1 - \cos \Delta \theta \right) - \frac{1}{r_0} - \frac{1}{r} \right] \\
\dot{g} &= 1 - \frac{\mu r_0}{h^2} \left( 1 - \cos \Delta \theta \right) \;,
\end{align}
$$ (lagrange)

where $r_0 = |\vec{r}_0| = \sqrt{\vec{r}_0 \cdot \vec{r}_0}$ is the magnitude of the initial position vector,
$v_0 = |\vec{v}_0| = \sqrt{\vec{v}_0 \cdot \vec{v}_0}$ is the magnitude of the initial velocity vector,
$v_{r0} = \vec{r}_0 \cdot \vec{v}_0 / r_0$ is the radial component of the initial velocity vector (the projection along the apse line),
$h = |\vec{h}|$ is the magnitude of the specific angular momentum,
and the new radial distance is

$$
r = \frac{h^2}{\mu} \frac{1}{1 + \left( \frac{h^2}{\mu r_0} - 1 \right) \cos \Delta \theta - \frac{h v_{r0}}{\mu} \sin \Delta \theta} \;.
$$

```{note}
While examples may show elliptical orbits, this approach applies regardless of orbit.
```

With these equations, we can now solve the problem.

## Example workflow

The workflow for solving a problem follows these steps:

1. Calculate the magnitudes and necessary quantities: $r_0 = |\vec{r}_0|$, $v_0 = |\vec{v}_0|$, 
$\vec{h} = \vec{r} \times \vec{v}$, $h = |\vec{h}|$, and $v_{r0} = \frac{ \vec{r}_0 \cdot \vec{v}_0 }{ |r_0| }$.

2. An optional step is to determine the initial true anomaly, and from that the eccentricity of the orbit $e$ 
(which would tell us what kind of orbit this is):

$$
\begin{align}
\theta_0 &= \tan^{-1} \left( \frac{r_{0y}}{r_{0x}} \right) \\
r_0 &= \frac{h^2 / \mu}{1 + e \cos \theta_0 } \\
\rightarrow e &= \left( \frac{h^2 / \mu}{r_0} - 1 \right) / \cos \theta_0
\end{align}
$$

3. Calculate the new radial distance:

$$
 r = \frac{h^2}{\mu} \frac{1}{1 + \left( \frac{h^2}{\mu r_0} - 1 \right) \cos \Delta \theta - \frac{h v_{r0}}{\mu} \sin \Delta \theta }
$$

4. Calculate the Lagrange coefficients $f, g, \dot{f}, \dot{g}$, using Equation {eq}`lagrange`.

5. Finally, calculate the new position and velocity vectors:

$$
\begin{align}
\vec{r} &= f \vec{r}_0 + g \vec{v}_0 \\
\vec{v} &= \dot{f} \vec{r}_0 + \dot{g} \vec{v}_0
\end{align}
$$

Here's example code in Matlab and Python for doing these steps:

````{tabbed} matlab

```{code-block} matlab

mu = 398.6e3; % km^3/s^2
r0 = [. . .];
v0 = [. . .];
delta_theta = .;

% 1. Calculate magnitudes
r0_mag = sqrt(dot(r0, r0));
v0_mag = sqrt(dot(v0, v0));

h = cross(r0, v0);
h_mag = sqrt(dot(h, h));
v_r0 = sqrt(dot(r0, v0)) / r0_mag;

% 2. Get eccentricity
theta_0 = atan(r0(2) / r0(1));
e = ((h_mag^2/(mu * r0_mag)) - 1) / cos(theta_0);

% 3. Get new distance
r_mag = (h_mag^2/mu) / (1 + ((h_mag^2/(mu*r0_mag)) - 1)*cos(delta_theta) - h_mag*v_r0 * sin(delta_theta)/mu);

% 4. Get Lagrange coefficients
f = 1 - mu * r_mag * (1 - cos(delta_theta)) / h_mag^2;
g = r_mag * r0_mag * sin(delta_theta) / h_mag;
fdot = (mu / h_mag)*(1 - cos(delta_theta))*( mu*(1 - cos(delta_theta))/h_mag^2 - (1/r0_mag) - (1/r_mag)) / sin(delta_theta);
gdot = 1 - mu*r0_mag*(1 - cos(delta_theta)) / h_mag^2;

% 5. Calculate new vectors:
r = f*r0 + g*v0;
v = fdot*r0 + gdot*v0;

```
````

````{tabbed} python

```{code-block} python
import numpy as np

mu = 398.6e3 # km^3/s^2
r0 = np.array([. . .])
v0 = np.array([. . .])
delta_theta = .

# 1. Calculate magnitudes
r0_mag = np.sqrt(np.dot(r0, r0))
v0_mag = np.sqrt(np.dot(v0, v0))

h = np.cross(r0, v0)
h_mag = np.sqrt(np.dot(h, h))
v_r0 = np.sqrt(np.dot(r0, v0)) / r0_mag

# 2. Get eccentricity
theta_0 = np.arctan(r0[1] / r0[0])
e = ((h_mag^2/(mu * r0_mag)) - 1) / np.cos(theta_0)

# 3. Get new distance
r_mag = (h_mag^2/mu) / (
    1 + ((h_mag^2/(mu*r0_mag)) - 1)*np.cos(delta_theta) - h_mag*v_r0 * np.sin(delta_theta)/mu
    )

# 4. Get Lagrange coefficients
f = 1 - mu * r_mag * (1 - np.cos(delta_theta)) / h_mag^2
g = r_mag * r0_mag * np.sin(delta_theta) / h_mag
fdot = (mu / h_mag)*(1 - np.cos(delta_theta))*(
    mu*(1 - np.cos(delta_theta))/h_mag^2 - (1/r0_mag) - (1/r_mag)
    ) / np.sin(delta_theta)
gdot = 1 - mu*r0_mag*(1 - np.cos(delta_theta)) / h_mag^2

# 5. Calculate new vectors:
r = f*r0 + g*v0
v = fdot*r0 + gdot*v0
```
````